# Factor models and expected returns

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px


Pull factor data

In [2]:
# Pull data
ff3 = pdr("F-F_Research_Data_Factors", "famafrench", start=1926)
ff5 = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=1964)

ff3_monthly = ff3[0]/100
ff5_monthly = ff5[0]/100

ff3_annual  = ff3[1]/100
ff5_annual  = ff5[1]/100

Estimate factor risk premia as time-series average return (using annual data)

In [3]:
# Annual 3 factors from 1926
fprem = ff3_annual[["Mkt-RF", "SMB", "HML"]].mean()

# Add annual RNW and CMA factors from 1964
fprem = pd.concat((fprem, ff5_annual[["RMW", "CMA"]].mean()))
factors = fprem.index.to_list()
fprem = fprem.round(4)
fprem

Mkt-RF    0.0855
SMB       0.0292
HML       0.0444
RMW       0.0353
CMA       0.0375
dtype: float64

Estimate factor loadings

In [4]:
# Function to fetch a yahoo time-series
def returns(ticker):
    ret = yf.download(ticker, start='2000-01-01', progress=False)
    ret.index = ret.index.to_period('D')
    ret = ret["Adj Close"].resample("M").last()
    ret = ret.pct_change()
    # ret.index = ret.index.to_period("M")
    ret.name = 'ret'
    return ret

In [5]:
# Pull stock returns
TICKER = 'IBM'
ret = returns('IBM')

d:\bbcx-investments-quarto\quarto\venv\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [6]:
# Add in factor returns and risk-free rate
df = pd.merge(ret,ff5_monthly, how="left",left_index=True,right_index=True)
df = df.dropna()
df

,ret,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2000-02,-0.083669,0.0245,0.1834,-0.0970,-0.1873,-0.0036,0.0043
2000-03,0.152068,0.0520,-0.1535,0.0817,0.1182,-0.0165,0.0047
2000-04,-0.058079,-0.0640,-0.0501,0.0726,0.0766,0.0565,0.0046
2000-05,-0.036394,-0.0442,-0.0384,0.0481,0.0417,0.0130,0.0050
2000-06,0.020966,0.0464,0.0993,-0.0843,-0.0832,-0.0293,0.0040
...,...,...,...,...,...,...,...
2022-09,-0.075049,-0.0935,-0.0097,0.0006,-0.0151,-0.0084,0.0019
2022-10,0.163959,0.0783,0.0186,0.0805,0.0307,0.0652,0.0023
2022-11,0.089560,0.0460,-0.0267,0.0138,0.0601,0.0311,0.0029


In [7]:
# Calculate excess return
df['xret'] = df.ret - df.RF 


In [8]:
# Estimate factor loadings using last 60 months
result = sm.OLS(df.ret.iloc[-60:], sm.add_constant(df[factors].iloc[-60:])).fit()
betas = result.params[1:]
betas = np.round(betas, 2)
betas

Mkt-RF    0.82
SMB       0.17
HML       0.02
RMW       0.53
CMA       0.69
dtype: float64

Calculate expected return using 
1. estimated factor loadings (`betas`)
2. estimated factor risk premia (`fprem`)
3. risk-free rate (`rf`)

In [9]:
# Most recent risk-free rate (in decimal notation)
rf = pdr("DGS3MO", "fred", start=1920).iloc[-1]/100
rf = rf[0]
rf

0.0473

In [10]:
# Expected return
expret = rf + betas @ fprem
print(f'The estimated expected return for {TICKER} is:\t {expret: .1%}')

The estimated expected return for IBM is:	  16.8%
